In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [3]:
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
# model = ChatOpenAI()
chain = prompt | model

In [4]:
chain.invoke({"foo": "bears"})

"Why don't bears use public restrooms?\\n\\nBecause they're afraid of bear traps!"

Often times we want to attach kwargs that’ll be passed to each model call. Here are a few examples of that:

## Attaching Stop Sequences

In [5]:
chain = prompt | model.bind(stop=["\n"])

In [6]:
chain.invoke({"foo": "bears"})

'Why did the bear get a ticket? Because he was caught texting while walking!'

## Attaching Function Call information

In [7]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [8]:
chain.invoke({"foo": "bears"}, config={})

"Why don't bears use computers?\\n\\nBecause they're afraid of bearware!"

## PromptTemplate + LLM + OutputParser
We can also add in an output parser to easily transform the raw LLM/ChatModel output into a more workable format

In [9]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [10]:
chain.invoke({"foo": "bears"})

'Why did the bear get a ticket? Because he was caught bear-itting on private property! 🐻������kok'

## Functions Output Parser
When you specify the function to return, you may just want to parse that directly

In [11]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [12]:
chain.invoke({"foo": "bears"})

OutputParserException: This output parser can only be used with a chat generation.

In [13]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [14]:
chain.invoke({"foo": "bears"})

OutputParserException: This output parser can only be used with a chat generation.

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | StrOutputParser(key_name="setup")
)

In [16]:
chain.invoke("bears")

"Why don't bears use computers?\\n\\nBecause they're bear-proof!"

In [17]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | StrOutputParser(key_name="setup")
)

In [18]:
chain.invoke("bears")

'Why did the bear get a ticket?\\n\\nBecause he was caught fishing without a license!'